In [10]:
import numpy as np
import torch 
from torch import nn
import calibration as cal
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained('vectara/hallucination_evaluation_model')
tokenizer = AutoTokenizer.from_pretrained('vectara/hallucination_evaluation_model')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
pairs = [
    ["A man walks into a bar and buys a drink", "A bloke swigs alcohol at a pub"],
    ["A person on a horse jumps over a broken down airplane.", "A person is at a diner, ordering an omelette."],
    ["A person on a horse jumps over a broken down airplane.", "A person is outdoors, on a horse."],
    ["A boy is jumping on skateboard in the middle of a red bridge.", "The boy skates down the sidewalk on a blue bridge"],
    ["A man with blond-hair, and a brown shirt drinking out of a public water fountain.", "A blond drinking water in public."],
    ["A man with blond-hair, and a brown shirt drinking out of a public water fountain.", "A blond man wearing a brown shirt is reading a book."],
    ["Mark Wahlberg was a fan of Manny.", "Manny was a fan of Mark Wahlberg."], 
]

inputs = tokenizer.batch_encode_plus(pairs, return_tensors='pt', padding=True)
print(inputs.keys())


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [15]:
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits.cpu().detach().numpy()
    print(logits)
    # convert logits to probabilities
    scores = 1 / (1 + np.exp(-logits)).flatten()
    print(scores)

[[ 0.4494805]
 [-7.651853 ]
 [ 5.6212516]
 [-8.457695 ]
 [ 5.5160384]
 [-6.56084  ]
 [-5.86595  ]]
[6.1051571e-01 4.7493709e-04 9.9639291e-01 2.1221573e-04 9.9599433e-01
 1.4126982e-03 2.8263179e-03]


In [13]:
sigmoid = nn.Sigmoid()
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    print(outputs)
    logits = outputs.logits
    print(logits)
    # convert logits to probabilities
    scores = sigmoid(logits)
    print(scores)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.4495],
        [-7.6519],
        [ 5.6213],
        [-8.4577],
        [ 5.5160],
        [-6.5608],
        [-5.8660]]), hidden_states=None, attentions=None)
tensor([[ 0.4495],
        [-7.6519],
        [ 5.6213],
        [-8.4577],
        [ 5.5160],
        [-6.5608],
        [-5.8660]])
tensor([[6.1052e-01],
        [4.7494e-04],
        [9.9639e-01],
        [2.1222e-04],
        [9.9599e-01],
        [1.4127e-03],
        [2.8263e-03]])


In [6]:
for name, module in model.named_modules():
    print(name)


deberta
deberta.embeddings
deberta.embeddings.word_embeddings
deberta.embeddings.LayerNorm
deberta.embeddings.dropout
deberta.encoder
deberta.encoder.layer
deberta.encoder.layer.0
deberta.encoder.layer.0.attention
deberta.encoder.layer.0.attention.self
deberta.encoder.layer.0.attention.self.query_proj
deberta.encoder.layer.0.attention.self.key_proj
deberta.encoder.layer.0.attention.self.value_proj
deberta.encoder.layer.0.attention.self.pos_dropout
deberta.encoder.layer.0.attention.self.dropout
deberta.encoder.layer.0.attention.output
deberta.encoder.layer.0.attention.output.dense
deberta.encoder.layer.0.attention.output.LayerNorm
deberta.encoder.layer.0.attention.output.dropout
deberta.encoder.layer.0.intermediate
deberta.encoder.layer.0.intermediate.dense
deberta.encoder.layer.0.intermediate.intermediate_act_fn
deberta.encoder.layer.0.output
deberta.encoder.layer.0.output.dense
deberta.encoder.layer.0.output.LayerNorm
deberta.encoder.layer.0.output.dropout
deberta.encoder.layer.1
deb